# Exploring Text on Maps

This notebook provides some examples of how to load and visualise the text outputs of MapReader. 

We focus on the geogrpaphy of railspace and labels describing railspace on maps.

----

In [1]:
import pandas as pd
import geopandas as geopd
import plotly.express as px
from collections import Counter
from tqdm import tqdm

# Load data

In [ ]:
# Load the patch predictions for railspace and railspace + buildings (codes 01 and 03)
predictions = geopd.read_file("/Users/kasparbeelen/Documents/LwM/imago-mundi/mapreader_maptext_outputs/MapReader_Data_SIGSPATIAL_2022/outputs/label_01_03/pred_01_03_keep_01_0250.csv")
predictions.head(3)

In [ ]:
# load the gepjson with spotted text
# we use the file that has been converted to point data
spotted_text = geopd.read_file("/Users/kasparbeelen/Documents/LwM/imago-mundi/mapreader_maptext_outputs/geo_predictions_deduplicated_point.json")
spotted_text.to_crs(epsg=27700, inplace=True)
spotted_text.shape

In [ ]:
spotted_text.head(3)

In [5]:
spotted_text['text_cleaned'] = spotted_text['text'].apply(lambda x: x.lower().strip().replace("(", "").replace(")", ""))

## Filtering labels

Here we discard the following labels
- those starting and ending with #
- those starting < or ending with >
- numbers after stripping the dot

The we lowercase all labels.

In [6]:
# filter spotted to reduce noise labels
filter_labels = lambda w : (w.endswith('#') or w.startswith('#') or w.endswith('>') or w.startswith('<') or w.strip('.').isdigit())
spotted_text_filtered = spotted_text[~spotted_text.apply(lambda x: filter_labels(x['text_cleaned']), axis=1)]

In [ ]:
# remove duplicates
spotted_text_filtered = spotted_text_filtered.drop_duplicates(subset=['patch_id','geometry','text']).reset_index(drop=True)

spotted_text_filtered.shape

In [ ]:
# print the shape of the dataframes
print(f"Railspace predictions shape: {predictions.shape}")
print(f"Spotted text original shape: {spotted_text.shape} filtered {spotted_text_filtered.shape}")

In [ ]:
spotted_text_filtered.text_cleaned.value_counts().head(10)

## Filtering patches

In [ ]:
# Retain patch predictions for maps that are in the spotted text data
text_map_ids = list(spotted_text.image_id.unique())
print('number of maps', len(text_map_ids))
# filter to those maps for which we have spotted text
predictions_red = predictions[predictions['parent_id'].isin(text_map_ids)]
predictions_red.shape,predictions.shape


In [ ]:
# add geometry to the patch predictions
predictions_red['geometry'] = geopd.points_from_xy(predictions_red.center_lon, predictions_red.center_lat)
predictions_red.reset_index(drop=True, inplace=True)

In [ ]:
predictions_red.head()

In [ ]:
# convert the projection to the same as the spotted text
predictions_red.crs = "epsg:4326"
predictions_red.to_crs(epsg=27700, inplace=True) # 27700


In [ ]:
spotted_text[spotted_text.distance(predictions_red.iloc[1001].geometry) < 100]

In [16]:
import xyzservices as xyz
tiles = xyz.TileProvider(
    name="OS 2nd Edition - 6 inch",
    url="https://api.maptiler.com/tiles/uk-osgb1888/{z}/{x}/{y}?key=5f6FYax2HhTa0Z9RfXsp",
    attribution="NLS",
)

In [ ]:
# double check by eyeballing some results
# Important: you might have to install a few packages to get this to work
# check the error message if it does not work
spotted_text[spotted_text.distance(predictions_red.iloc[121].geometry) < 100].explore(tiles=tiles)

## Collect labels close to railspace

The below cell identifies the text that falls within the 100m of railspace patches. Text within this distance is stored as "adjacent text" and any other text is stored as "other text".

You can **skip** this cell if you don't want to change the setting and go the next cell where we load the railspace labels which we saved in a text file.

In [ ]:
tqdm.pandas()
adjacent_text = [] # here we store labels close to the target category, i.e. railspace

distance = 100 # maximum distance in meters between patch centroid and text centroid

for i,row in tqdm(predictions_red.iterrows(), total=predictions_red.shape[0]):
    # get text within a certain distance from the patch centroid
    # get the set of text labels
    labels = spotted_text_filtered[spotted_text_filtered.distance(row.geometry) <= distance].text_cleaned.tolist()
    adjacent_text.extend(labels)

# save the adjacent text to a txt file
with open('adjacent_txt.txt', 'w') as out_txt:
    out_txt.write('\n'.join(adjacent_text))

print('Railspace labels',len(adjacent_text))

In [ ]:
spotted_text.head()

In [ ]:
adjacent_text[:10]

# Important!
Skip to this cell if you don't want to run the code for finding adjacent text. 


We just load the railspace labels as a text file.

In [ ]:
with open('adjacent_txt.txt', 'r') as in_txt:
    adjacent_text = [i.strip() for i in in_txt.readlines()]
adjacent_text[:10], len(adjacent_text)

## Compute proportional difference 

We compute the proportional difference between the labels on railspace patch versus the 'background' i.e. all labels on the maps.

In [19]:
# get the text labels on the maps
all_text = spotted_text_filtered.text_cleaned.tolist()

In [20]:
# get counts and probabilities for the text labels for the railspace category
railspace_text_freq =  Counter([i.lower() for i in adjacent_text])
railspace_text_prob = {k: v/ sum(railspace_text_freq.values()) for k,v in railspace_text_freq.items()} 

In [21]:
# get counts and probabilities of the text labels for all the maps
all_text_freq =  Counter([i.lower() for i in all_text])
all_text_prob = {k: v/ sum(all_text_freq.values()) for k,v in all_text_freq.items()}

In [ ]:
# compare both absoluate counts and probabilities of a give word
word = 'railway'
print(railspace_text_freq[word], all_text_freq[word])
print(railspace_text_prob[word], all_text_prob[word])

In [23]:
# compute the proportional difference between the probabilities of the text labels 
# in railspace and all the maps
proportional_difference = sorted(
                                {w: railspace_text_prob.get(w,0) - all_text_prob.get(w,0) 
                                        for w in all_text_prob.keys()}.items(), 
                                    key=lambda x: x[1], 
                                    reverse=True
                                    )


In [ ]:
print('Railscape labels')
print(proportional_difference[:5])
print('Other labels')
print(proportional_difference[-5:])

## Plot proportional difference

In [ ]:
pd.DataFrame(proportional_difference[:20]).plot(kind='bar', x=0, y=1, legend=False, 
                            title='Top 20 terms in Building labels', 
                            xlabel='Term', ylabel='Difference in probability')

In [ ]:
pd.DataFrame(proportional_difference[-20:]).plot(kind='bar', x=0, y=1, legend=False, 
                            title='Top 20 terms in Other labels', 
                            xlabel='Term', ylabel='Difference in probability')

To get a sense of what some of the abbreviations mean, please go to the NLS website: https://maps.nls.uk/os/abbrev/

# Visalizing the semantic of text on maps

In the visualization below we encode each label to a vector using BERT-type language model. This generates a vector for each labels that approximates the 'meaning' of this label. Then we visualize these embeddigns in two dimensional space where you can explore the different semantic regions of the text data.

In [27]:
# uncomment the following line to run if you have not yet installed sentence-transformers, scikit-learn and plotly
#!pip install -U -q sentence-transformers scikit-learn plotly

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.manifold import TSNE
from sentence_transformers import SentenceTransformer
import plotly.express as px
from sklearn.metrics.pairwise import cosine_distances

In [ ]:
# get all text labels above a certain threshold to obtain the railspace labels
threshold = 0.0001 # i.e. labels with prob > threshold are in railspace
railspace_labels = [w for w,v in proportional_difference if v > threshold]
len(railspace_labels)

In [ ]:
# plot all the 'railway' labels on the map
spotted_text[spotted_text.text_cleaned.isin(railspace_labels)].explore(tiles=tiles)

In [ ]:
# get all other labels based on the inverse of the railspace threshold
 # i.e. labels with prob < -threshold are in the other category
other_labels = [w for w,v in proportional_difference if v < threshold*-1]
len(other_labels)

In [ ]:
# embed the railspace labels
# load pre-trained sentence transformer model
# you can plug in the BLERT model here but for TSNES we better use the distilbert model
model = SentenceTransformer('distilbert-base-nli-mean-tokens') # Livingwithmachines/bert_1760_1900

# encode the sentences
railspace_sentence_embeddings = model.encode(railspace_labels)

# perform dimensionality reduction using TSNE
tsne = TSNE(n_components=2, random_state=42)
embeddings_tsne = tsne.fit_transform(railspace_sentence_embeddings)

In [ ]:
# visualize the labels in 2D scatter plot using the probability difference as size
data = pd.DataFrame(embeddings_tsne, columns=['x','y'])
data['text'] = railspace_labels
data['size'] = [np.log(all_text_freq[w]) for w in railspace_labels]
fig = px.scatter(data, x="x", y="y", text='text', size='size', width=1000, height=1000,)
fig.show()

In [ ]:
# visualize the railspace and other labels
labels = railspace_labels + other_labels
sentence_embeddings = model.encode(labels)


tsne = TSNE(n_components=2, random_state=42)
embeddings_tsne = tsne.fit_transform(sentence_embeddings)

# visualize the labels in 2D scatter plot using the probability difference as size
data = pd.DataFrame(embeddings_tsne, columns=['x','y'])
data['color'] = ['railspace']* len(railspace_labels) + ['other']* len(other_labels)
data['text'] = labels
data['size'] = [np.log(all_text_freq[w]) for w in labels]
fig = px.scatter(data, x="x", y="y", text='text', size='size', color='color', width=1000, height=1000,)
fig.show()

In [ ]:
# plot all labels higher than a certain threshold
freq_threshold = 5
labels = [w for w in list(set(all_text)) if all_text_freq[w]> freq_threshold]; print(len(labels))
sentence_embeddings = model.encode(labels)

tsne = TSNE(n_components=2, random_state=42)
embeddings_tsne = tsne.fit_transform(sentence_embeddings)

# visualize the labels in 2D scatter plot using the probability difference as size
data = pd.DataFrame(embeddings_tsne, columns=['x','y'])
data['color'] = ['railspace' if w in railspace_labels  else 'other' for w in labels]
data['text'] = labels
data['size'] = [np.log(all_text_freq[w]) for w in labels]
fig = px.scatter(data, x="x", y="y", text='text', size='size', color='color', width=1000, height=1000,)
fig.show()

## Semantice Distance

In [51]:
# load the embedding model
model = SentenceTransformer('distilbert-base-nli-mean-tokens') # Livingwithmachines/bert_1760_1900 | distilbert-base-nli-mean-tokens
# encode the railspace labels
railspace_sentence_embeddings = model.encode(railspace_labels)
# create a railspaces embedding, feel free to add other labels
railspace = model.encode(['rail railway'])


In [52]:
# compute the cosine distance between the railspaces labels and the labels
data = pd.DataFrame(railspace_labels,columns=['text'])
data['sem_dist_to_rail'] = cosine_distances(railspace_sentence_embeddings, railspace.reshape(1, -1))

In [ ]:
data.sort_values('sem_dist_to_rail').head(10)

In [ ]:
data.sort_values('sem_dist_to_rail').tail(10)

In [ ]:
# encode the other labels
other_sentence_embeddings = model.encode(other_labels)
# what are the closest labels to the railspace embedding
pd.DataFrame.from_records([other_labels,
              list(cosine_distances(other_sentence_embeddings, railspace.reshape(1, -1)).reshape(-1))],
              ).T.sort_values(1).head(10)


# Clustering experiments

In [ ]:
from sklearn.cluster import KMeans
import json
import numpy as np

In [85]:
# important this cell takes a while to run
# you can simply ignore and load the json file I've shared with you
tqdm.pandas()
adjacent_texts = [] # here we store labels close to the target category, i.e. railspace

distance = 250 # maximum distance in meters between patch centroid and text centroid

for i,row in tqdm(predictions_red.iterrows(), total=predictions_red.shape[0]):
    # get text within a certain distance from the patch centroid
    # get the set of text labels
    labels = list(set(spotted_text_filtered[spotted_text_filtered.distance(row.geometry) <= distance].text_cleaned.tolist()))
    labels = sorted(labels)
    # add the labels as a list sorted alphabetically
    adjacent_texts.append(labels)


with open('adjacent_texts.json', 'w') as out_txt:
    json.dump(adjacent_texts, out_txt)

100%|██████████| 26835/26835 [56:59<00:00,  7.85it/s]    


In [106]:
# load the labels from json
with open('adjacent_texts.json', 'r') as in_txt:
    adjacent_texts_json = json.load(in_txt)


In [86]:
# filter the list to remove short and non-alphabetic labels
# this is not ideal as it remove abbreviations and other useful information
# especially in the more denser urban areas, however we use it as a simple experiment
# if you want to keep all labels, just comment out the following line
labels_sentence = [' '.join([w for w in a if (len(w) > 2) and w.isalpha()]) for a in adjacent_texts]
# and uncomment the following line
# labels_sentence = [' '.join([w for w in a]) for a in adjacent_texts]

In [87]:
# load the embedding model
model = SentenceTransformer('distilbert-base-nli-mean-tokens') # Livingwithmachines/bert_1760_1900 | distilbert-base-nli-mean-tokens
# encode the railspace "sentences", i.e. the list of alphabetically sorted labels
railspace_sentence_embeddings = model.encode(labels_sentence)

In [93]:
railspace_sentence_embeddings.shape, predictions_red.shape

((26835, 768), (26835, 19))

In [94]:

# fit the kmeans model to the railspace embeddings
# we use here 10 clusters, feel free to change the number of clusters
kmeans = KMeans(n_clusters=10, random_state=0, n_init="auto").fit(railspace_sentence_embeddings)


In [100]:
# add labels to the predictions_red dataframe
predictions_red['cluster'] = kmeans.labels_
predictions_red['labels'] = labels_sentence

/Users/kasparbeelen/anaconda3/envs/mapreader/lib/python3.9/site-packages/geopandas/geodataframe.py:1528: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

/Users/kasparbeelen/anaconda3/envs/mapreader/lib/python3.9/site-packages/geopandas/geodataframe.py:1528: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [111]:
# plot the different clusters on the map
predictions_red[[ 'geometry', 'cluster', 'laberels']].explore(column='cluster', tiles=tiles)

# Plot labels/sentences in semantic space

In [102]:
tsne = TSNE(n_components=2, random_state=42)
embeddings_tsne = tsne.fit_transform(railspace_sentence_embeddings)
embeddings_tsne

array([[ 78.65548 , -73.02942 ],
       [ 23.085203, -60.233402],
       [ 78.75422 , -72.95126 ],
       ...,
       [ 60.639362,   7.259105],
       [ 67.37914 ,   7.27327 ],
       [ 67.30581 ,   7.260923]], dtype=float32)

In [112]:
data = pd.DataFrame(embeddings_tsne, columns=['x','y'])
data['text'] = labels_sentence
fig = px.scatter(data, x="x", y="y", text='text', width=1000, height=1000,)
fig.show()

# Fin.